# Synthetic data generator for deep calibration

In this notebook we generate the synthetic labeled data needed for training the neural networks.

In [ ]:
# Standard library imports
import numpy as np
import pandas as pd
import string
import sys

Setting the graphics environment.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter

def set_style():

    sns.set_context("paper")

    sns.set(font='serif')
    
    sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]
    })

set_style()
%matplotlib inline

## Joint distribution of (moneyness,time_to_maturity)

Data generation is an expensive and time-consuming task. Since we want our neural network to be as accurate as possible in the parameter region of greatest liquidity, it makes sense to provide more training data in that region by sampling more labeled data from it. Specifically, we will hence compute an empirical joint probability distribution of (moneyness, time_to_maturity) from SPX data where liquidity is proxied by *open interest*.

First, we get settlement prices for call and put options on the S&P 500 index from www.cboe.com/DelayedQuote/QuoteTableDownload.aspx .

In [ ]:
# Market parameters
spot_price = 2731.25
today = pd.to_datetime('20180215', format='%Y%m%d', errors='coerce')
nb_trading_days = 365

# Read in raw SPX data 
df = pd.read_csv('raw_data/spx_20180215.csv', skiprows=2)

### Data preprocessing

We discard all information about puts and also some columns that are of no interest in this task.

In [ ]:
# Remove information about puts
df = df.iloc[:,:7]

# Drop superfluous columns
del_cols = ['Last Sale', 'Net', 'Vol', 'Bid', 'Ask']
df.drop(del_cols, axis=1, inplace=True)

In [ ]:
def extract_info(data):
    
    # Create a dictionary for conversion of month code (A-L: Jan-Dec for 
    # call, M-X: Jan-Dec for put) to MM
    letters = [letter for letter in string.ascii_uppercase[:24]]
    numbers = [nb for nb in range(1, 13)] + [nb for nb in range(1, 13)]
    cal_dict = dict(zip(letters,numbers))
    
    # Identify ticker from content
    ticker = str(data).split(' ')[3][1:-1]
    
    # Is option European or American?
    is_euro = True if ticker[-1] == 'E' else False
    
    # Is option Weekly?
    is_weekly = True if ticker[3] == 'W' else False
    
    # Differentiate between variables ticker lengths for Weekly and 
    # Standard options
    ticker = ticker[4:] if is_weekly else ticker[3:]
    
    # Detect time to maturity in years from ticker symbol.
    yy = '20'+ ticker[0:2]
    dd = ticker[2:4]
    mm = str(cal_dict[ticker[4]]).zfill(2)
    timestamp = pd.to_datetime(yy+mm+dd, format='%Y%m%d', errors='coerce')
    dtm = (timestamp - today).days
    dtm_frac = dtm/nb_trading_days
    
    # Detect moneyness from ticker symbol.
    strike = ticker[5:9]
    cont = True
    i = 0

    while cont:
        cont = strike[i].isdigit()
        if cont == False:
            strike = strike[:i]
        else:
            i += 1
            if i >= len(strike):
                break
    
    moneyness = spot_price/int(strike)
    
    return is_euro, is_weekly, dtm_frac, moneyness

Extract information from ticker symbol and add to dataframe.

In [ ]:
# Extract information from ticker and write to separate df
df2 = df['Calls'].apply(extract_info).apply(pd.Series)
df2.columns = ['is_euro', 'is_weekly', 'time to maturity (years)', 
               'moneyness']

# Merge two dataframes
df = pd.concat([df, df2], axis=1)

# Drop the now redundant column 'Calls'
df.drop(['Calls'], axis=1, inplace=True)

Next, we only consider European options and SPX Weeklys which expire every Monday, Wednesday and Friday. 

In [ ]:
df = df[df['is_euro'] == True]
df = df[df['is_weekly'] == True]
df.drop(['is_euro'], axis=1, inplace=True)
df.drop(['is_weekly'], axis=1, inplace=True)

Naturally, those options with 0 *Open interest* are of no interest to us, so drop them as well.

In [ ]:
df = df[df['Open Int'] != 0]
df.sort_values('moneyness', inplace=True, ascending=False)

Possibility to export dataframe to csv file.

In [ ]:
df.to_csv('raw_data/processed_spx_data.csv')

In the following plot we can see that large parts of the parameter region are empty with no data.

In [ ]:
oi_pivot_df = df.pivot(index='time to maturity (years)', 
                       columns='moneyness', values='Open Int')

ax = sns.heatmap(oi_pivot_df, cmap="Blues", cbar=True, xticklabels=15, yticklabels=2)
ax.invert_yaxis()

plt.tight_layout()
fig = plt.gcf()
fig.savefig('raw_data/full_moneyness_ttm_heatmap.pdf')

We thus reduce the parameter region considered to $0.75\leq moneyness \leq 1.2$ and $0\leq time to maturity \leq 0.25$ and plot another heatmap.

In [ ]:
df = df[df['moneyness'] <= 1.2]
df = df[df['moneyness'] >= 0.75]
df = df[df['time to maturity (years)'] <= 0.25]

In [ ]:
oi_pivot_df = df.pivot(index='time to maturity (years)', 
                       columns='moneyness', values='Open Int')

ax = sns.heatmap(oi_pivot_df, xticklabels=10, cmap="BuPu")
ax.invert_yaxis()

plt.tight_layout()
fig = plt.gcf()
fig.savefig('raw_data/reduced_moneyness_ttm_heatmap.pdf')

### KDE Estimation of (Moneyness, Time to Maturity)

As discussed, our goal is to sample from $(moneyness, time to maturity)$ and more crucially, to sample more from parameter regions with higher liquidity as proxied by *Open interest*. The idea is to fit a Kernel Density Estimation to our data using the package ScikitLearn and then generate new samples from it. Before we can do that, we need to preprocess our data however. Since KDE cannot weigh the individual pairs by *Open interest*, the hack is to create a new df where each row is duplicated until the number of rows with that data point is equal to its *Open interest*. 

In [ ]:
# Reindexing dataframe so that it starts from 1
df = df.reset_index(drop=True)

# Initiating a new df of appropriate size
total_interest = df['Open Int'].sum()
kde_df = pd.DataFrame(index = np.arange(0, total_interest), 
                      columns=['time to maturity (years)', 'moneyness'], 
                      dtype='float64')

# Filling the new df with entries according to their multiplicities 
# proxied by open interest
kde_index = 0

for i in df.index:
    
    mult = df.loc[i, 'Open Int']
    values = [df.loc[i, ['time to maturity (years)', 'moneyness']]]*mult
    kde_df.loc[kde_index:kde_index + mult-1] = values
    
    kde_index += mult  

#### Approach I : Seaborn visualization
Seaborn is a visualization library and includes a KDE Plot. 

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Bivariate KDE of moneyness and time to maturity')
x = kde_df['moneyness']
y = kde_df['time to maturity (years)']
ax = sns.kdeplot(x, y, cbar=True, shade=True, shade_lowest=False, cmap='BuPu')
fig.savefig("sns_kde_money_maturities_V3.pdf", dpi=400)

The KDE plot shows what one might expect from the heatmap before. Unfortunately, Seaborn does not return the actual KDE model it computes in its backend, so we also can't sample from it. Digging in the source code reveals that it uses - if installed - the statsmodels KDE estimator. Otherwise it defaults to the scipy KDE estimator. So our next try will be to use the statsmodel implementation.

#### Approach II: Statsmodels

In [ ]:
from statsmodels.nonparametric.kernel_density import KDEMultivariate

In [ ]:
X = kde_df[['moneyness', 'time to maturity (years)']]
kde = KDEMultivariate(X, 'cc', bw='normal_reference')
kde

This algorithm is very fast but it lacks two important features:
1. There is no method to sample from the distribution.
2. There is no chance to include finite bounds for $(K,T)$ which is especially bad for $T$ as the KDE would span over the negative domain as well, losing probability mass over a nonsensible domain.

#### Approach III: Scikit-Learn KDE
Unlike statsmodels, this KDE estimator does have a method to sample from but just like statsmodels there is no possibility to include bounds.

In [ ]:
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
import statsmodels.nonparametric.api as smnp

data = kde_df[['moneyness', 'time to maturity (years)']].values

Two choices for bandwidth selection. Either Scott's rule or Cross validation. Run only one cell.

In [ ]:
# Scott's rule (rule of thumb) for bandwidth selection
bw_x = smnp.bandwidths.bw_scott(df['moneyness'])
bw_y = smnp.bandwidths.bw_scott(df['time to maturity (years)'])

print('bw_x: {}, bw_y: {}'.format(bw_x, bw_y))

kde = KernelDensity(bandwidth=max(bw_x, bw_y))

In [ ]:
# use grid search cross-validation to optimize the bandwidth
params = {'bandwidth': np.logspace(-3, -1, 5)}
grid = GridSearchCV(KernelDensity(), params, verbose=sys.maxsize, n_jobs=-1)
grid.fit(data)

print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))

kde = grid.best_estimator_

Run KDE Estimation of Data.

In [ ]:
kde.fit(data)

Naturally, the KDE extends to regions outside the input region we have previously considered. That's why we will use rejection sampling below to compute our new samples.
Remember we want $0.75\leq moneyness \leq 1.2$ and $0\leq time to maturity \leq 0.25$.

In [ ]:
nb_samples = 10**6
new_data = pd.DataFrame(index=np.arange(10**6),
                        columns=['moneyness', 'time to maturity (years)'],
                        dtype='float64')

valid_counter = 0

while valid_counter < nb_samples:
    
    rem = nb_samples - valid_counter
    
    # Generate new samples from estimated probability density.
    raw = kde.sample(rem)
    
    # Identify valid samples
    is_valid = (raw[:,0]>0.75) & (raw[:,0]<1.2) & (raw[:,1]>0) & (raw[:,1]<0.25)
    valid_samples = raw[is_valid]
    nb_valid_samples = valid_samples.shape[0]
    
    # Writing to df
    new_data.loc[valid_counter: valid_counter + nb_valid_samples - 1 ,:] = valid_samples

    valid_counter += nb_valid_samples
    
# Save (moneyness,time to maturity (years)) to disk
filepath = 'raw_data/money_maturities.csv'
new_data.to_csv(filepath)

## Synthetic generation of BS Implied Volatility data

In [ ]:
from scipy.stats import norm

In [ ]:
def pricer(flag, spot_price, strike, time_to_maturity, vol, risk_free_rate):
    """
    Computes the Black-Scholes price of a European option with possibly
    vectorized inputs.

    :param flag: either "c" for calls or "p" for puts
    :param spot_price: spot price of the underlying
    :param strike: strike price
    :param time_to_maturity: time to maturity in years
    :param vol: annualized volatility assumed constant until expiration
    :param risk_free_rate: risk-free interest rate

    :type flag: string
    :type spot_price: float / numpy array
    :type strike: float / numpy array
    :type time_to_maturity: float / numpy array
    :type vol: float / numpy array
    :type risk_free_rate: float

    :return: Black-Scholes price
    :rtype: float / numpy array

    # Example taking vectors as inputs

    >>> spot = np.array([3.9,4.1])
    >>> strike = 4
    >>> time_to_maturity = 1
    >>> vol = np.array([0.1, 0.2])
    >>> rate = 0

    >>> p = BlackScholes.pricer('c', spot, strike, time_to_maturity, vol, rate)
    >>> expected_price = np.array([0.1125, 0.3751])


    >>> abs(expected_price - p) < 0.0001
    array([ True,  True], dtype=bool)
    """

    # Rename variables for convenience.

    S = spot_price
    K = strike
    T = time_to_maturity
    r = risk_free_rate

    # Compute option price.

    d1 = 1/(vol * np.sqrt(T)) * (np.log(S) - np.log(K) + (r+0.5 * vol**2) * T)

    d2 = d1 - vol * np.sqrt(T)

    if flag == 'c':

        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    elif flag == 'p':

        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return price

def write_to_csv(file_name, data, header):

        np.savetxt(file_name, data, delimiter=',', newline='\n', header=header)

Now, let's generate our labeled data. Note that the number of samples of the precomputed bivariate data of (moneyness, time to maturity) has to equal the number of desired samples below.

In [ ]:
# Declare seed for sampling from parameter regions.
np.random.seed()

nb_samples = 10**6

# Model parameters
spot = 1           # spot
rate = 0           # rate
flag = 'c'      # 'c' for call
vol_lower = 0.01
vol_higher = 1

# Reading and transforming precomputed data
input_data = np.load('money_maturities.npy')
moneyness, maturities = input_data[:,0], input_data[:,1]
strikes = spot/moneyness

# Sample uniformly from input space.
vols = np.random.uniform(vol_lower, vol_higher, nb_samples)

# Compute Black-Scholes price. 
prices = pricer(flag, spot, strikes, maturities, vols, rate)

In [ ]:
# Create labeled pair and randomly permute rows
labeled_data = np.stack([moneyness, maturities, prices, vols], axis=1)
labeled_data = np.random.permutation(labeled_data)

# Dissecting labeled pairs into training, validation and testing sets.
training_data = labeled_data[:8*10**5,:]
validation_data = labeled_data[8*10**5:9*10**5,:]
testing_data = labeled_data[9*10**5:, :]
print('Shapes: train {}, valid {}, test {}'.format(training_data.shape, validation_data.shape,
                                                  testing_data.shape))

# Write labeled data to .csv file.
header = 'moneyness, maturities, prices, vols'
write_to_csv('training_data_new.csv', training_data, header)
write_to_csv('validation_data_new.csv', validation_data, header)
write_to_csv('testing_data_new.csv', testing_data, header)

## Synthetic generation of Heston data

In [ ]:
import rpy2
import rpy2.robjects as robjects
import numba
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_lets_be_rational.exceptions import BelowIntrinsicException
import sklearn.utils

r = robjects.r
r.source("heston.R")
r_pricer = r('HestonCallClosedForm')

In [ ]:
def heston_pricer(lambd, vbar, eta, rho, v0, r, tau, S0, K):
    """
    Computes European Call price under Heston dynamics with closedform solution.

    :param lambd: mean-reversion speed
    :param vbar: long-term average volatility
    :param eta: volatility of vol
    :param rho: correlation between stock and vol
    :param v0: intial volatility
    :param r: risk-free interest rate
    :param tau: time to maturity (year = 365 days)
    :param S0: initial share price
    :param K: strike price

    :return: Heston price, Black-Scholes implied volatility
    :rtype: float, float

    """
    
    try:
        price = r_pricer(lambd, vbar, eta, rho, v0, r, tau, S0, K)[0]

        try:

            iv = implied_volatility(price, S0, K, tau, r, 'c')

        except BelowIntrinsicException:

            print('Below Intrinsic Exception with parameters:', lambd, vbar, 
                   eta, rho, v0, r, tau, S0, K)

            iv = None

    except rpy2.rinterface.RRuntimeError:

        print('R Runtime Error with parameters:', lambd, vbar, eta, rho, 
              v0, r, tau, S0, K)

        price, iv = None, None

    return price, iv

In [ ]:
# Declare seed for sampling from parameter regions.
np.random.seed()

nb_samples = 10**6

# Standardised parameters
S0 = 1
r = 0

# Heston parameter bounds by Moodley (2005)
lambd_bounds = [0, 10]
vbar_bounds = [0, 1]
eta_bounds = [0, 5]
rho_bounds = [-1, 0]
v0_bounds = [0, 1]

# Import and initialising of data
df = pd.read_csv('raw_data/money_maturities.csv')
moneyness = df['moneyness'].values
maturity = df['time to maturity (years)'].values
strike = S0/moneyness

columns = ['lambda', 'vbar', 'eta', 'rho', 'v0', 'maturity', 'moneyness', 'iv']
df = pd.DataFrame(np.zeros((nb_samples,8)), columns=columns)

# Counter that tracks the amount of computed labeled pairs
count = 0

Simulation of Heston labeled pairs.

In [ ]:
with open("heston_errors.csv", "w") as log:

    log.write("lambda, vbar, eta, rho, v0, maturity, moneyness, price, \
               intrinsic, price < intrinsic, abs(price) < 1E-5, OK?  \n" )
    
    while count < nb_samples:

        # Sample uniformly from Heston parameter space.
        lambd = np.random.uniform(lambd_bounds[0], lambd_bounds[1])
        vbar = np.random.uniform(vbar_bounds[0], vbar_bounds[1])
        eta = np.random.uniform(eta_bounds[0], eta_bounds[1])
        rho = np.random.uniform(rho_bounds[0], rho_bounds[1])
        v0 = np.random.uniform(v0_bounds[0], v0_bounds[1])

        # Take respective (strike, maturity) pair from precomputed data.
        K = strike[count]
        T = maturity[count]
        M = moneyness[count]

        # Calculate Black-Scholes implied vol from Heston price.
        price, iv = heston_pricer(lambd, vbar, eta, rho, v0, r, T, S0, K)
        
        # Running through possible cases returned by heston_pricer.
        if iv is not None:
            
            df.loc[count, columns] = [lambd, vbar, eta, rho, v0, T, M, iv]
            
            print('Count {}/{}'.format(count + 1, nb_samples))

            # Increase running counter.
            count += 1
            
        elif price is not None:

            # Collect all necessary information to judge why it failed.
            error_data = (lambd, vbar, eta, rho, v0, T, M, 
                          price, np.max(S0 - K, 0), price < np.max(S0 - K), 
                          np.abs(price) < 1E-5, 
                          (price < np.max(S0 - K)) or (np.abs(price) < 1E-5))

            log.write("%s \n" % repr(error_data))
            
        else:

            error_data = (lambd, vbar, eta, rho, v0, T, M)

            log.write("%s \n" % repr(error_data))

In [ ]:
df = sklearn.utils.shuffle(df)

# Dissecting labeled pairs into training, validation and testing sets.
train_df, validation_df, test_df = np.split(df, [8*10**5, 9*10**5], axis=0)

print('Shapes: \n train {}, validation {}, test {}'.format(train_df.shape, validation_df.shape,
                                                  test_df.shape))

# Write labeled data to .csv file.
train_df.to_csv('heston/training_data.csv', index=False)
validation_df.to_csv('heston/validation_data.csv', index=False)
test_df.to_csv('heston/testing_data.csv', index=False)